In [108]:
import pyspark
from pyspark import SparkContext

In [2]:
sc = SparkContext(appName="task6")

In [5]:
INPUT_FILE_PATH = "Rec1_F19_Files_to_zip/file01_Hd_Sp_Freq"

# MapReduce Functions

In [114]:
lines = sc.textFile(INPUT_FILE_PATH)
linesRef = sc.textFile(INPUT_FILE_PATH)
linesRef = linesRef.collect()

def getRidOfDupes(termTuple):
    key = termTuple[0]
    occurrences =  list(map(lambda x: (x[0],x[1]),termTuple[1]))
    termIndices = []
    for i in occurrences:
        termIndices.append(i[0])
    firstLineOccurrence = min(termIndices)
    inLineOccurrences = []
    for j in occurrences:
        if j[0] == firstLineOccurrence :
            inLineOccurrences.append(j[1])
    firstInLineOccurrence = min(inLineOccurrences)
    return key,(firstLineOccurrence,firstInLineOccurrence)


def mapLineWithLineIndex(line):
    for i in range(len(linesRef)):
        if line == linesRef[i]:
            return (line,i)

def splitLineWithLineIndex(line):
    r"""
        "line" here is a tuple with two elements.
        line[0] = the actual line
        line[1] = index of the line
    """
    tokenized = line[0].split(" ")
    outputList = []
    for i in range(len(tokenized)):
        token = tokenized[i]
        outputList.append((token.lower(),(line[1],i)))
    return outputList

def reAlignByLines(token):
    word = token[0]
    line = token[1][0]
    postitionInLine = token[1][1]
    return line,(word,postitionInLine)

def mergeLineContent(line):
    r"""
        "line" here is the following:
            - line[0] = the line number
            - line[1] = iterator that has elements: tuple(word,positionInLine)
                - in the following function x[0] = word and x[1]= positionInLine
    """
    lineNum = line[0]
    wordList = list(map(lambda x: (x[0],x[1]),line[1]))
    wordList.sort(key=lambda x: x[1])
    sortedWordList = list(map(lambda kvPair: kvPair[0],wordList))
    return lineNum,sortedWordList

def reduceToSolution(acc,current):
    stringList_acc = acc[1]
    stringList_acc.append('\n')
    stringList_current = current[1]
    for i in stringList_current:
        stringList_acc.append(i)
    return acc


noDupes = lines.map(lambda line: mapLineWithLineIndex(line)).flatMap(splitLineWithLineIndex).groupByKey().map(getRidOfDupes)\
                .map(reAlignByLines).groupByKey().sortByKey().map(mergeLineContent).reduce(reduceToSolution)

noDupesRdd = sc.parallelize(noDupes[1])
noDupesRdd.saveAsTextFile("Problem06_NoDuplicates_Output")
print(noDupesRdd.collect())

['hello', 'world', 'bye', 'how', 'are', 'you', 'what', 'is', 'your', 'name', '', '\n', 'i', 'study', 'spark', 'and', 'also', 'hadoop', '\n', 'prefer', 'to', '\n', 'but', 'have', 'spend', 'more', 'hours', 'studying', '\n', 'difficult', 'compile', 'debug', 'some', 'say', '\n', 'am', 'interested', 'in', 'marreduce', 'which', 'part', 'of', '\n', 'so', 'shall', 'choose', '\n', 'perhaps', 'can', 'compromise', "let's", 'forget', 'about', 'it', 'all', 'do', 'sql', '\n', 'cloud', 'computing', '\n', 'supposed', 'get', 'familiar', 'with', 'nosql', 'pighive', '\n', 'much', '\n', 'oh', 'it.', "i'll", 'only', 'as', 'this', 'started', '\n', 'no,', "don't", 'want', 'be', 'modern?', 'then', "i'd", 'better']


# Output

In [115]:
#format the output to match the input format:
outptuString = ""
for token in noDupesRdd.collect():
    outptuString += token + " "
print(outptuString)

hello world bye how are you what is your name  
 i study spark and also hadoop 
 prefer to 
 but have spend more hours studying 
 difficult compile debug some say 
 am interested in marreduce which part of 
 so shall choose 
 perhaps can compromise let's forget about it all do sql 
 cloud computing 
 supposed get familiar with nosql pighive 
 much 
 oh it. i'll only as this started 
 no, don't want be modern? then i'd better 


In [116]:
sc.stop()